### All Techniques of Hyper Parameter Optimization

1. GridSearchCV
2. RandomizedSearchCV
3. Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
4. Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)
5. Optuna- Automate Hyperparameter Tuning
6. Genetic Algorithms (TPOT Classifier)

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings (action= 'ignore')
plt.style.use(['seaborn-bright' , 'dark_background'])

In [21]:
data = pd.read_csv("diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [22]:
data['Glucose'] = np.where(data['Glucose'] == 0 , data['Glucose'].median() , data['Glucose'])
data['Insulin'] = np.where(data['Insulin'] == 0 , data['Insulin'].median() , data['Insulin'])
data['SkinThickness'] = np.where(data['SkinThickness'] == 0 , data['SkinThickness'].median() , data['SkinThickness'])
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [23]:
X = data.iloc[: , :-1]
Y = data.iloc[: , -1]

In [24]:
from sklearn.model_selection import train_test_split as tts
x_train , x_test , y_train , y_test = tts(X, Y , test_size=0.2 , random_state=33)

In [25]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators= 10)
classifier.fit(x_train , y_train)

RandomForestClassifier(n_estimators=10)

In [26]:
Y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [27]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix , accuracy_score
C_M = confusion_matrix(y_test , y_pred)
print(C_M)
print(accuracy_score(y_test , y_pred))

[[86 13]
 [32 23]]
0.7077922077922078


### Hyperparameter Tuning Techniques

### Random Grid 

In [28]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200 , 2000 , 10)]
# Number of features to consider at every split 
max_features = ['auto' , 'sqrt' , 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10 , 1000 , 10)]
# Maximum number of samples required to split a node
min_samples_split = [2 , 5, 10 , 14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6 , 8]
# Create the Random Grid
random_grid = {'n_estimators' : n_estimators,
               'max_features' : max_features,
               'max_depth' : max_depth,
               'min_samples_split' : min_samples_split,
               'min_samples_leaf' : min_samples_leaf,
               'criterion' : ['entropy' , 'gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


### Randomized Search CV

In [29]:
from sklearn.model_selection import RandomizedSearchCV
Random_Search_CV = RandomizedSearchCV(estimator= RandomForestClassifier() , param_distributions= random_grid , n_iter = 10 , cv = 2 , verbose = 2 , random_state= 100 , n_jobs = -1) 
Random_Search_CV.fit(x_train , y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=1000, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END criterion=entropy, max_depth=1000, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.3s


/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=6, min_samples_split=14, n_estimators=200; total time=   0.3s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=6, min_samples_split=14, n_estimators=200; total time=   0.3s
[CV] END criterion=gini, max_depth=670, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.8s
[CV] END criterion=gini, max_depth=670, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.8s
[CV] END criterion=gini, max_depth=120, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1400; total time=   1.6s
[CV] END criterion=gini, max_depth=120, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1400; total time=   1.7s


/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=340, max_features=sqrt, min_samples_leaf=1, min_samples_split=14, n_estimators=2000; total time=   2.3s
[CV] END criterion=gini, max_depth=340, max_features=sqrt, min_samples_leaf=1, min_samples_split=14, n_estimators=2000; total time=   2.4s
[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=14, n_estimators=2000; total time=   2.6s
[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=14, n_estimators=2000; total time=   2.5s
[CV] END criterion=gini, max_depth=340, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   1.6s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=600; total time=   0.6s
[CV] END criterion=gini, max_depth=340, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   1.7s
[CV] END criterion=ent

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [30]:
Random_Search_CV.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 120,
 'criterion': 'entropy'}

In [31]:
Random_Search_CV.best_score_

0.7752442996742671

### Param Grid

In [32]:
param_grid = {
    'criterion' : [Random_Search_CV.best_params_['criterion'] , 'entropy'],
    'max_depth' : [Random_Search_CV.best_params_['max_depth']],
    'max_features' : [Random_Search_CV.best_params_['max_features']],
    'min_samples_leaf' : [Random_Search_CV.best_params_['min_samples_leaf'],
                          Random_Search_CV.best_params_['min_samples_leaf'] + 2,
                          Random_Search_CV.best_params_['min_samples_leaf'] + 4],
    'min_samples_split' : [Random_Search_CV.best_params_['min_samples_split'] - 2,
                           Random_Search_CV.best_params_['min_samples_split'] + 1,
                           Random_Search_CV.best_params_['min_samples_split']],
    'n_estimators' : [Random_Search_CV.best_params_['n_estimators'] - 200,
                      Random_Search_CV.best_params_['n_estimators'] + 100,
                      Random_Search_CV.best_params_['n_estimators']]
}
print(param_grid)

{'criterion': ['entropy', 'entropy'], 'max_depth': [120], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [0, 3, 2], 'n_estimators': [400, 700, 600]}


### Grid Search CV

In [33]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator= RandomForestClassifier() , param_grid = param_grid , cv = 2 , n_jobs = -1 , verbose = 2)
grid_search.fit(x_train , y_train)

Fitting 2 folds for each of 54 candidates, totalling 108 fits
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=400; total time=   0.1s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=400; total time=   0.1s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=700; total time=   0.2s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=600; total time=   0.1s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=600; total time=   0.2s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_split=0, n_estimators=700; total time=   0.2s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=1, min_samples_spli

GridSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'entropy'],
                         'max_depth': [120], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [0, 3, 2],
                         'n_estimators': [400, 700, 600]},
             verbose=2)

In [34]:
grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 120,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 400}

In [35]:
grid_search.best_score_

0.7801302931596091

In [36]:
me = RandomForestClassifier(n_estimators = 200, min_samples_split= 4, min_samples_leaf =  5, max_features = 'log2', max_depth = 500 , criterion = 'entropy')
me.fit(x_train , y_train)

RandomForestClassifier(criterion='entropy', max_depth=500, max_features='log2',
                       min_samples_leaf=5, min_samples_split=4,
                       n_estimators=200)

In [37]:
y_me = me.predict(x_test)
accuracy_score(y_test , y_me)

0.7532467532467533

### Automated Hyperparameter Tuning 
Automated Hyperparameter Tuning can be done by using techniques such as

* Bayesian Optimization
* Gradient Descent
* Evolutionary Algorithms

### Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

* Objective Function = defines the loss function to minimize.
* Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
* Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [38]:
from hyperopt import hp , fmin , tpe , STATUS_OK , Trials

In [39]:
space = {
         'criterion' : hp.choice('criterion' , ['entropy' , 'gini']),
         'max_depth' : hp.quniform('maxdepth', 10 , 1200 , 10),
         'max_features' : hp.choice('max_features', ['auto' , 'sqrt' , 'log2' , None]),
         'min_samples_leaf' : hp.uniform('min_samples_leaf', 0 , 0.5),
         'min_samples_split' : hp.uniform('min_samples_split', 0 , 1),
         'n_estimators' : hp.choice('n_estimators', [10 , 50 , 300 , 750 , 1200])
        }

* hp.choice -> For Selecting 
* hp.quniform -> For Integers
* hp.uniform -> For Floating numbers 

In [40]:
from sklearn.model_selection import cross_val_score
def objective(space) : 
    model = RandomForestClassifier(criterion= space['criterion'] , max_depth = space['max_depth'],
        max_features= space['max_features'] , min_samples_leaf= space['min_samples_leaf'],
        min_samples_split= space['min_samples_split'],
        n_estimators= space['n_estimators'])
    
    accuracy = cross_val_score(model , x_train , y_train , cv = 5).mean()

    # We aim to maximize accuracy , therefore we return it as a negaitve value
    return {'loss' : -accuracy , 'status' : STATUS_OK}


In [41]:
trail = Trials()
# fmin minimizes the function values
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 80,
            trials= trail)

best

  0%|          | 0/80 [00:00<?, ?trial/s, best loss=?]

job exception: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 476, in fit
    trees = Parallel(
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/joblib/parallel.py", line 90

  0%|          | 0/80 [00:00<?, ?trial/s, best loss=?]


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 476, in fit
    trees = Parallel(
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/kushalbanda/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/joblib/parallel.py", line 901, 

In [ ]:
trained_forest = RandomForestClassifier(criterion= 'gini' , max_features= None,
max_depth= 160 , min_samples_split= 0.0032947923955915018,
min_samples_leaf= 0.035426869579208625, n_estimators= 2)
trained_forest.fit(x_train , y_train)

RandomForestClassifier(max_depth=160, max_features=None,
                       min_samples_leaf=0.035426869579208625,
                       min_samples_split=0.0032947923955915018, n_estimators=2)

In [ ]:
y_pred = trained_forest.predict(x_test)
accuracy_score(y_test , y_pred)

0.7467532467532467